In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader

file_path = (
    "/Users/hcaap/Desktop/Kod/Siemens/Trying_rag/hållflösningar.pdf"
)

loader = UnstructuredPDFLoader(file_path)
data = loader.load()

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(data)

In [18]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings

ollama_emb = OllamaEmbeddings(
    model="mxbai-embed-large",
)

db = Chroma.from_documents(documents, ollama_emb)

In [29]:
query = "What did the author say about the importance of innovation in the business world?"
docs = db.similarity_search_with_score(query, k=4)
docs

[(Document(page_content='For instance, the theory of disruptive innovation, another concept coined by Christensen and discussed in Disruptive Strategy, is the process by which a smaller company—usually with fewer resources—moves upmarket and challenges larger, established businesses.\n\nBy mastering theories like this one, you can help your organization strategize how to disrupt incumbent companies or prepare for emerging competitors.\n\nAsking yourself the questions in this section can shed light on whether you want to start a business. If you’ve decided that you’re ready to become an entrepreneur, there are certain skills you need to start your journey on the right path. Those are covered next.\n\nDeciding to Pursue Entrepreneurship 14\n\nSkills You Need to Be an Innovative Entrepreneur\n\nUnlike personality traits and demographic details, entrepreneurship and innovation skills can be learned and practiced. As you embark on your entrepreneurial journey, focus your efforts on developi

In [26]:
retriever = db.as_retriever()

In [37]:
retriever = db.as_retriever(k=4)
docs = retriever.invoke("what are they key concepts for innovation?")



In [30]:
import ollama

prompt = "What are key concepts for innovation in business?"


output = ollama.generate(
  model="llama3",
  prompt=f"Using this data: {docs}. Respond to this prompt: {prompt}"
)

print(output['response'])

Based on the provided data, some key concepts for innovation in business include:

1. **Disruptive Innovation**: The process by which a smaller company with fewer resources can move upmarket and challenge larger, established businesses.
2. **Structured Experimentation**: Conducting market research and running tests to determine whether new opportunities are worth pursuing.
3. **Long-Term Focus**: Entrepreneurship is a long-term endeavor that requires focusing on the process from beginning to end to achieve long-term success.
4. **Innovation Culture**: Embracing innovation as a characteristic of some entrepreneurs, which can be cultivated and developed over time.
5. **Experimentation and Learning**: Running experiments early in the entrepreneurial journey to learn and adapt to changing market needs.
6. **Continuous Improvement**: Fostering an environment that encourages continuous improvement, iteration, and learning.

These concepts suggest that innovation in business requires a combin

In [34]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context. 
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""

from langchain_community.llms import Ollama

llm = Ollama(model="llama3")


question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(llm, question_answering_prompt)

In [38]:
from langchain_core.messages import HumanMessage

document_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content="What are som key things to become and entrepreneur?")
        ],
    }
)

'According to the context, some key skills you need to be an innovative entrepreneur include:\n\n1. Ability to learn concepts through case studies\n2. Mastering theories like disruptive innovation\n3. Being able to accept and act on feedback\n4. And possibly others (the list is not exhaustive)\n\nNote that these skills can be learned and practiced, making entrepreneurship a feasible career path for many individuals.'